<a href="https://colab.research.google.com/github/matthewchung74/thisbonedoesnotexist/blob/main/medical_text_generation_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

based on the work of https://huggingface.co/pucpr/gpt2-bio-pt

In [ ]:
!pip install -q datasets transformers[sentencepiece] sacrebleu sacremoses
!pip install -q gputil
!pip install -q psutil
!pip install -q humanize
!pip install -q sentence_transformers

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import torch
import re

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def translate(input_text):
  print(f'using device')
  # https://stackoverflow.com/questions/52208602/regex-for-splitting-into-sentences-ignoring-decimal-numbers-as-part-of-the-spli
  sub_str = '[\.\!]+(?!\d)\s*|\n+\s*'
  input_text_arr = re.split(sub_str, input_text)
  output_text_arr = []

  src_text_arr = [f">>por<< {text}" for text in input_text_arr]

  model_name = "Helsinki-NLP/opus-mt-roa-en"
  tokenizer = MarianTokenizer.from_pretrained(model_name)
  ['>>zlm_Latn<<', '>>mfe<<', '>>hat<<', '>>pap<<', '>>ast<<', '>>cat<<', '>>ind<<', '>>glg<<', '>>wln<<', '>>spa<<', '>>fra<<', '>>ron<<', '>>por<<', '>>ita<<', '>>oci<<', '>>arg<<', '>>min<<']

  model = MarianMTModel.from_pretrained(model_name).to(device)

  for src_text in src_text_arr:
    if src_text.strip() == '>>por<<':
      continue

    # print(f'source {src_text}')
    src_text = src_text.replace(" cm", " centimeter")
    model.generate(**tokenizer(src_text, return_tensors="pt", padding=True).to(device))
    translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True).to(device)).to(device)
    # print(f"using model device {model.device} translated device {translated.device}")
    output_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    # print(f'translated {output_text}')
    if output_text is not None and len(output_text) > 0:
      output_text_arr.append(output_text[0])

  return '. '.join(output_text_arr)

In [ ]:
from transformers import pipeline

chef = pipeline('text-generation',model="pucpr/gpt2-bio-pt", tokenizer="pucpr/gpt2-bio-pt")

reports = []
for i in range(100):
  reports.append(translate(chef('vista frontal e lateral do tórax')[0]['generated_text']))

reports

In [ ]:
import re

new_reports = []
consequitivedots = re.compile(r'\.{3,}')

for report in reports:
  new_reports.append(consequitivedots.sub('', report))

In [ ]:
import numpy as np
np.savetxt('reports.csv', reports, delimiter=',', fmt='%s')

In [ ]:
from google.colab import files
files.download('reports.csv') 